## ReMKiT1D input generator - building fluid models using ReMKiT1D's custom model feature

ReMKiT1D allows for the construction of models using a granular approach, where the user has a high level of control without having to modify the Fortran code. 

This is accomplished through custom model objects, and this notebook is meant to serve as a detailed tutorial and documentation for the fluid custom model case. 

The problem dealt with will be a 2-fluid system, fully coupling ions and electrons with Bohm outflow boundary conditions and with particle and energy sources. The method used is a collocated finite volume method, and the ringing that results from this can be seen in the solution. However, the point of this notebook is to provide detailed examples of how one would go about doing practically relevant model construction using ReMKiT1D. Staggered grid examples for both fluid and kinetic models are available in other notebooks, where comments are sparser.


The following are dependencies for this example

In [1]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds
import RMK_support.common_models as cm

### Some useful constants

In [2]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27 #atomic mass unit
ionMass = 2.014*amu # deuterium mass
epsilon0 = 8.854188e-12 #vacuum permittivity 

### Wrapper initialization

In [3]:
rk = RKWrapper()

### Global parameters for IO files

The default value for the ReMKiT1D json input is config.json, and most ReMKiT1D input routines search for this file, unless a different file name is specified in the command line.
HDF5 files are by default also assumed to be in the local directory.

In [4]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_custom_fluid/" 
rk.setHDF5Path(hdf5Filepath)

### Normalization

The default normalization object in ReMKiT1D contains the following values

1. density (in $m^{-3}$) - $n_0$ input. Defaults to 1e19. 
2. temperature (in eV) - $T_0$ input. Defaults to 10. Accessed as "eVTemperature"
3. reference ion Z - input. Defaults to 1. Accessed as "referenceIonZ"
4. velocity (used for the velocity grid) - $v_{th}=\sqrt{m_eeT_0/2}$ using normalization temperature. Accessed as "velGrid"
5. speed (here equal to the velocity) - $u_0$. 
6. time (normalized to e-i collision time) - $t_0$. 
7. length ($t_0 v_{th}$). 
8. EField (here $m_e v_{th}/(et_0)$)).
9. heatFlux (here $m_e n_0 v_{th}^3/2$). 
10. crossSection (here $1/(n_0t_0v_{th}$)

The default inputs (1-3) can be changed by setting the following:

In [5]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

### Grid initialization

The ReMKiT1D grid ecosystem consists of the following objects: 

- grid data: contains raw grid cell centre coordinates and harmonic data  
- geometry data: contains x space cell widths and Jacobian value
- velocity space data: contains velocity space cell widths and moment taking support

The initialization of standard versions of these objects is greatly simplified, requiring the following data to initialize
the Grid object:

-  **xGrid**: a numpy.ndarray representing either spatial cell centre coordinates or cell widths to be used to determine cell centres
-  **vGrid**: like the above, but for the velocity grid
-  **lMax**: the highest resolved spherical harmonic (Legendre) l number relevant only when distributions are used
-  **mMax**: like the above, but for the m number. Defaults to 0. Note that this is not used in the current version of the code and is implemented for future-proofing.
-  **interpretXGridAsWidths**: if True interprets xGrid as cell widths. Defaults to False.
-  **interpretVGridAsWidths**: if True interprets vGrid as cell widths. Defaults to False.
-  **isPeriodic**: if True sets spatial grid to be periodic. Defaults to False. 
-  **isLengthInMeters**: True if xGrid values are assumed unnormalized, i.e. in meters. Defaults to False.

The default face Jacobian is unity, but this can be changed with the Grid.xJacobian setter

NOTE: ReMKiT1D automatically generates a staggered/dual grid for the supplied x-grid by taking the right cell faces as cell centres of the dual grid. Variables can then be set to live on the dual grid. While arrays of the same length are used to store both regular and staggered grid variables, on non-periodic grids the last value on a staggered grid is ignored. This way the staggered grid represents interior cell faces while outer cell faces (where boundary conditions are required) are treated in different ways. 

Below is an example of a grid initialization with a uniform grid in X and logarithmic grid in velocity space widths. The velocity grid is not used in this example as all equations are only fluid equations.

In [6]:
xGrid = 0.1*np.ones(128) # Total length of 12.8m
vGrid = np.logspace(-2,1,80) #In normalized velocity - default normalization is thermal velocity sqrt(m_e * k * T_e/2)
lMax = 0
gridObj = Grid(xGrid,vGrid,lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True,isLengthInMeters=True)
gridObj.xJacobian = np.ones(129) #The face jacobian must be one element longer than the grid object to account for external faces. Here set to default value.


In [7]:
# Add the grid to the wrapper
rk.grid = gridObj

### Handling particle species data

A number of ReMKiT1D objects require a list of species (see standard textbook below for example). 
Species are identified by their name and an integer ID. The following convention is used for the IDs:

- 0 for electrons (enforced at the Fortran level)
- negative for other charged species (regardless of charge)
- positive for neutral species

The following species properties can be set:

1. atomic mass (in amus)
2. species charge (in elementary charge)
3. associated variables - list of variable names associated with this species - some features can use this list to streamline model assembly

In the following example two species are added, one electron and one ion species.

**NOTE**: ReMKiT1D will always set the ID=0 species mass and charge to the electron values, regardless of the input. 

In [8]:
rk.addSpecies("e",0,associatedVars=["ne","Ge","We"]) #Adding associated variables allows some models to automatically detect implicit and evolved variables - this is unused in this example
rk.addSpecies("D+",-1,atomicA=2.014,charge=1.0,associatedVars=["ni","Gi","Wi"])

### Textbook objects and derivations

In the variable container example above, we have added derived quantities to the variable container, but have not specified how to calculate them. In ReMKiT1D this is handled through derivation objects - function wrappers operating on variables. In general derivations are stored in a textbook object, and require a list of variables to be used with. 

An example of a ReMKiT1D textbook is the standard textbook object. It contains the following default derivations:

1. "flowSpeedFromFlux" - 2 input variables interpreted as flux and density, returns flux/density (normalized to $n_0 u_0$)
2. "sonicSpeed" for each ion species  (negative IDs) - 2 input variables interpreted as $T_e$ and $T_i$. Returns $\sqrt{(\delta_e  T_e + \delta_i T_i)/m_i}$ where i is the index of the ion species (normalized to $u_0$). $\delta_e$ and $\delta_i$ can be specified (see below).
3. "tempFromEnergy" for each species specified (see below) - three input variables interpreted as energy, density, and flux. Returns 2/3 * energy/density - mass * flux $^2$/(3 * density $^2$), where mass refers to the species mass.
4. "leftElectronGamma" - 2 input variables interpreted as the $T_e$ and $T_i$ used to calculate the sheath electron heat transmission coefficient at the left boundary
5. "rightElectronGamma" - 2 input variables interpreted as the $T_e$ and $T_i$ used to calculate the sheath electron heat transmission coefficient at the right boundary
6. "densityMoment" - single input variable interpreted as electron distribution. Returns zeroth moment of $f_0$.
7. "energyMoment" - single input variable interpreted as electron distribution. Returns second order of $f_0$/2.

If harmonic $l=1$ is resolved:

8. "fluxMoment" - single input variable interpreted as electron distribution. Returns first moment of $f_1$/3 (normalized to $n_0u_0$)
9. "heatFluxMoment" - single input variable interpreted as electron distribution. Returns third moment of $f_1$/3 (normalized to $m_e  n_0 v_{th}^3 / 2$)

If harmonic $l=2$ is resolved:

10. "viscosityTensorxxMoment" - single input variable interpreted as electron distribution. Returns second moment of $2f_2/15$ (normalized to $n_0 e T_0$)

Interpolation for staggered grids: 

11. "gridToDual" = interpolates one variable from regular to staggered(dual) grid
12. "dualToGrid" = interpolates one variable from staggered(dual) to regular grid

Other useful derivations:

13. "gradDeriv" = calculates gradient of variable on regular grid, interpolating on cell faces and extrapolating at boundaries if needed
14. "logLei" = calculates electron-ion Coulomb log taking in electron temperature and density (derivations added for each ion species)
15. "logLee" = calculates electron self collision Coulomb log taking in electron temperature and density 
16. "logLii" = calculates ion-ion collision frequency for each ion collision combination taking in the two species temperatures and 
              densities (used as"logLiis_S" where s and S are the first and second ion species name) 
17. "maxwellianDistribution" = calculates distribution with Maxwellian f0 taking in electron temperature and density


The following dictionary structure can be used to set standard textbook values:

In [9]:
rk.setStandardTextbookOptions([-1,0],ePolyCoeff=1.0,ionPolyCoeff=1.0,electronSheathGammaIonSpeciesID=-1) 

Custom derivations are by far the most common derivation types used with ReMKiT1D. These can be constructed using individual derivation types in simple_containers.py, or can be built using the ReMKiT1D expression tree structure. Examples of both are available in other notebooks.

### Variable container

Most variables in ReMKiT1D are stored in a VariableContainer object. 
Variables are defined by their name and the following:

1. fluid or distribution - determines whether the variable is defined on the velocity grid/harmonics
2. implicit or derived - implicit variables can be evolved using implicit methods, while derived variables can have derivation rules associated to them (see below)
3. whether or not they are stationary - stationary variables have d/dt = 0 
4. whether they are defined on the regular or dual grid
5. whether or not they are a scalar derived variable - time should be set as scalar
6. the variable priority - an integer where 0 is the highest priority and greater numbers correspond to lower priorities. Currently only relevant for derived variables with assigned derivation rules during integrations with internal stages. Defaults to 0.

### Variables

The following variables are added in this example.

Electron and ion conserved variables - implicit:

1. densities "ne" and "ni"
2. fluxes "Ge" and "Gi"
3. energy densities "We" and "Wi" 
4. temperatures "Te" and "Ti" - implicitly derived stationary variables

Stationary conductive heat fluxes:

5. "qe" and "qi" - both implicit

Electric field:

6. "E" - implicit

Derived fluid quantities: 

7. flow speeds "ue" and "ui" 
8. sonic speed "cs" 

Scalar quantities: 

9. default time variable "time"
10. left and right electron sheath heat transmission coefficient "gammaLeft" and "gammaRight"

Evaluation quantities: 

11. Qei - energy exchange term source in electron energy equation, obtained using evaluation type manipulator
11. logL - e-i Coulomb Logarithm obtained using a modelbound data extractor type manipulator

NOTE: Stationary implicit variables MUST have a model "evolving" them (see below), otherwise PETSc preconditioning will fail due to a zero pivot. 

NOTE: As mentioned above, ReMKiT1D models can treat variables as if they are defined on the staggered grid. This is specified at the point of defining models, and ReMKiT1D does NOT check for consistent usage. The example in this notebook does not use the staggered/dual grid.

In [10]:
n = np.ones(gridObj.numX())
T = np.ones(gridObj.numX())
W = 3*n*T/2
# Set conserved variables in container
rk.addVar("ne",n,units='$10^{19} m^{-3}$',isCommunicated=True) #Units are not used by ReMKiT1D, but are useful to specify for later plotting
rk.addVar("ni",n,units='$10^{19} m^{-3}$',isCommunicated=True)
rk.addVar("Ge",isCommunicated=True)
rk.addVar("Gi",isCommunicated=True)
rk.addVar("We",W,units='$10^{20} eV m^{-3}$',isCommunicated=True)
rk.addVar("Wi",W,units='$10^{20} eV m^{-3}$',isCommunicated=True)

# Set heat fluxes 

rk.addVar("qe",isStationary=True,isCommunicated=True)
rk.addVar("qi",isStationary=True,isCommunicated=True)

# Set E field

rk.addVar("E")

rk.addVar("Te",T,isStationary=True,units='$10eV$',isCommunicated=True)
rk.addVar("Ti",T,isStationary=True,units='$10eV$',isCommunicated=True)

# Set derived fluid quantities

rk.addVar("ue",isDerived=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["Ge","ne"]))
rk.addVar("ui",isDerived=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["Gi","ni"]))
rk.addVar("cs",isDerived=True,derivationRule=sc.derivationRule("sonicSpeedD+",["Te","Ti"]))
rk.addVar("Qei",isDerived=True)
rk.addVar("logL",isDerived=True)

# Set scalar quantities 
rk.addVar("time",isScalar=True,isDerived=True)
rk.addVar("gammaLeft",isScalar=True,isDerived=True,derivationRule=sc.derivationRule("leftElectronGamma",["Te","Ti"]),isCommunicated=True,hostScalarProcess=0)
rk.addVar("gammaRight",isScalar=True,isDerived=True,derivationRule=sc.derivationRule("rightElectronGamma",["Te","Ti"]),isCommunicated=True,hostScalarProcess=7)

ionGamma = 2.5*np.ones([1]) # Scalar variables must be specified as a length 1 numpy array
rk.addVar("ionGamma",ionGamma,isScalar=True,isDerived=True)

### Setting options for external libraries used by ReMKiT1D

ReMKiT1D uses a number of external library features which need to be set at runtime. The libraries in question are:

1. MPI - used for all communication
2. PETSc - used for performing linear solves using KSP methods by default
3. HDF5 - handles data output (and optionally input)

#### MPI

ReMKiT1D allows for parallelization in both x-space and the space of harmonics. The number of processes in each direction is specified by setting numProcsX and numProcsH and should conform with the grid and total number of processors used to run ReMKiT1D. 

There are three major classes of communication routines in ReMKiT1D:

1. Broadcasting along the h-processor direction
2. Halo exchange between x-processor neighbours
3. Broadcasting scalar variables to all processors from the processor responsible for the calculation

Halo size (in cells) can be set by changing the xHaloWidth variable. Default communication rules can also be specified using the commData field containing:

1. varsToBroadcast: names of variables that should be broadcast in the h direction (usually all variables)
2. haloExchangeVars: names of variables that participate in halo exchange (a good rule of thumb is to include all those variables that appear under spatial derivatives)
3. scalarVarsToBroadcast: names of scalars to be broadcast to all processes
4. scalarBroadcastRoots: root processors hosting the above scalar variables (e.g. those processes where the variable is accurately evolved)


In [11]:
numProcsX = 8 # Number of processes in x direction (do not change here, as the scalar processor roots are hard-coded in this example)
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells
rk.setMPIData(numProcsX,numProcsH,haloWidth)

#### PETSc

PETSc is used in ReMKiT1D to solve linear matrix equations using Krylov subspace methods. For more information see [PETSc documentation](https://petsc.org/release/docs/). 

Default wrapper values are used here.

#### HDF5

The HDF5 library is used for input and output of variable and grid data.

No HDF5 input is used here, and all variables in the wrapper are outputted by default.

### Models 

ReMKiT1D is organized into "models", modular collections of terms and supporting data allowing for highly flexible simulation setups. 

Firstly, a term is generally of the form $dv/dt = S$, where $v$ is the evolved variable. $S$ depends on the structure of the term, but is in principle arbitrary (depending only on the available term implementations). Two main classes of terms are matrix and explicit terms, with the following properties:

- matrix terms:  
    - represent a linearized matrix for use in implicit methods - currently only Backwards Euler with Picard iterations
    - must have a declared implicit variable - i.e. the term is of the form $dv/dt = Mu$, where $u$ is the implicit variable and both $v$ and $u$ are arrays representing variables after discretization
    - can be explicitly evaluated - i.e. can be used in explicit methods by just evaluating $Mu$ for a given set of variables
- explicit terms:
    - composed of a constant, a multiplicative function, and an operator - $dv/dt = cf(v,u,...)O*u$, where $u$ is now referred to as the operated variable
    - only usable in explicit methods (but generally easier to code and design)

Another important concept is that of grouping terms. This allows for selective evaluation of terms or sets of terms and for easy implementation of operator-splitting methods (see the integrator section below). There are two classes of term groups, implicit and general. 

The numbers of implicit and general groups is set in the integrator options, and groups are indexed in that order, i.e. general term group with index $i$ is term group with index size(implicitGroups) + $i$. 

Terms can belong to more than one group, which is useful for some applications and types of models.

Models are identified by tag and type, and the tag is used to specify individual model options. 

In this notebook, custom fluid models with matrix terms will be used to construct all used models. Each model will be added separately and explained.

### Structure of the custom fluid model

The custom fluid model is made of matrix terms and (optionally) modelbound data. 

Each term is identified by a tag used to provide it with options, and a model can have any number of terms. General terms are of the form $cX(x)T(t)M_{ij}u_j$, where i and j are row and column indices (summation implied) associated with the evolved and implicit variables. $X$ and $T$ are functions only of the spatial cell and the "time" variable, while $c$ is a constant. The main term matrix $M_{ij}$ is further assumed to be of the form $R_iC_jS_{ij}$, where $R$ and $C$ are row and column components which are simple products of variables raised to corresponding powers, e.g. $R = \prod_n v_n^{p_n}$ ($R$ can also contain evaluation results of other term groups - see below). Finally, the main structure of the term resides in the "stencil" component $S_{ij}$, the options for which will be reviewed below. For the currently implemented Picard iteration Backwards Euler implicit integrator, $T$ and $M$ are updated at each iteration using variable values from the previous one, while $u$ is fully implicit. 

The following is a list of common options for these general matrix terms:

1. "evolvedVar" = name of the implicitly evolved (row) variable - must be specified
2. "implicitVar" = name of implicit (column) variable - must be specified
3. "spatialProfile" = real array conforming to x-grid size ($X$ in the above). Defaults to array of ones.
4. "harmonicProfile" = real array conforming to h-grid size. Defaults to array of ones. Not used in fluid terms.
5. "velocityProfile" = real array conforming to v-grid size. Defaults to array of ones. Not used in fluid terms.
4. "evaluatedTermGroup" = optional term group index to be explicitly evaluated and used to multiply $R$. Defaults to 0, not evaluating any group
5. "implicitGroups" = list of implicit groups this term belongs to. Defaults to [1]
6. "generalGroups" = list of general groups this term belongs to. Defaults to [1]
7. "customNormConst" - options for the $c$ constant (which defaults to 1)
    1. "multConst" = multiplicative constant. Defaults to 1.
    2. "normNames" = names of normalization constants used to calculate $c$. Defaults to empty array
    3. "normPowers" = powers to raise the above normalization constants to. Defaults to array of ones conforming to the size of "normNames".
8. "timeSignalData" - options for the $T$ function in the above
    1. "timeSignalType" = name of the time signal function. Defaults to "none". This determines the shape of the signal. See the TimeSignal class in simple_container.py for more documentation.
    2. "timeSignalPeriod" = period of the T function 
    3. "timeSignalParams" = parameters used to calculate the T function (depend on the type of signal)
    4. "realTimePeriod" = if true interprets the "timeSignalPeriod" value in seconds instead of normalized time units
9. "varData" - options for the $R$ and $C$ components of $M$ in the above
    1. "requiredRowVarNames" and "requiredRowVarPowers" - variable names and powers used to calculate $R$
    2. "requiredColVarNames" and "requiredColVarPowers" - variable names and powers used to calculate $C$
    3. "requiredMBRowVarNames" and "requiredMBRowVarPowers" - modelbound variable names and powers used in calculating $R$
    4. "requiredMBColVarNames" and "requiredMBColVarPowers" - modelbound variable names and powers used in calculating $C$ 
10. "stencilData" - options for the $S$ component of $M$ - discussed in more detail below
    1. "stencilType" = type of the stencil, determines other used stencil data options 

Routines for easier construction of custom terms and models are supplied in simple_containers and are demonstrated below.

###  Stencil templates

For a list of currently implemented stencils see simple_containers.py

Some fluid stencils useful for this example are:

#### 1. Diagonal stencil

Used for terms local in space. 

If both the row and column variable live on the same (regular or staggered) grid, the stencil is truly diagonal. If they live on different grids,
the stencil is changed to implicitly interpolate (or extrapolate if near non-periodic boundary) the column variables onto the row variable's grid.

NOTE: Once again, the last "cell" on the staggered grid when the grid is not periodic is not evolved, as it would represent the boundary face. 

#### 2. Central difference stencil with interpolation on faces

Represents a finite difference stencil calculated by interpolating values on internal cell faces. This is a 3-point stencil which should in general be second order accurate. The following "stencilData" options are used by this stencil:

1. "ignoreJacobian" = true if the stored x-grid jacobian should be ignored and the used values set to 1. Useful when writing grad terms. Defaults to false
2. "interpolatedVarName" = name of the optional variable interpolated on cell faces. Useful for setting the flux jacobian in flux divergence terms. Defaults to "none"

Both the row and column variables must live on the same grid for this stencil.

#### 3. Staggered difference stancil 

Represents a finite difference stencil where the row and column variables live on different (regular vs staggered) grids. The used stencil is a 2-point stencil, forward difference if the row variables are staggered, and backwards if the column variables are on the staggered grid. Treats only internal points on non-periodic grids. The following "stencilData" options are used by this stencil:

1. "ignoreJacobian" = true if the stored x-grid jacobian should be ignored and the used values set to 1. Useful when writing grad terms. Defaults to false

#### 4. Upwinded difference

Represents an upwinded flux difference stencil based on the value of a specified flux jacobian variable on cell faces. Assumes that variables involved with the stencil are either all on the real or all on the staggered grid. The used stencil is dynamically updated depending on the sign of the flux on cell faces, but a 3-point stencil is allocated to avoid overhead. The following "stencilData" options are used by this stencil.

1. "ignoreJacobian" = true if stored x-grid jacobian should be ignored and the used values set to 1. Included for completeness. Defaults to false
2. "fluxJacVar" =  name of the flux jacobian variable to be interpolated on faces and used in constructing stencil values. Must be supplied 

Both row and column variables must live on the same grid for this stencil.

NOTE: This stencil can lead to problems, and staggered grids are preferred to upwinding at the time of writing this example. 

#### 5. Boundary condition stencil

Represents a boundary stencil for div- and grad-like operators. Allows to set a lower bound for an optionally extrapolated flux jacobian at the boundary face. However, even in the staggered grid case, it assumes that the flux jacobian and lower bound variables live on the regular grid. The following "stencilData" options are used by this stencil:

1. "ignoreJacobian" = true if the stored x-grid jacobian should be ignored and the used values set to 1. Useful when writing grad terms. Defaults to false
2. "leftBoundary" = true if the stencil is to be built for the left boundary. Defaults to false (builds right boundary stencil)
3. "dontExtrapolate" = true if the stencil is to be built to use the column values in the boundary cell without linearly extrapolating to the physical boundary. Defaults to false
4. "noLowerBound" = set to true if no lower bound is to be set to the extrapolated flux jacobian. Defaults to false
5. "fluxJacVar" = optional flux jacobian to be extrapolated to the boundary. Defaults to "none" 
6. "lowerBoundVar" = optional lower boundary variable to be extrapolated to the boundary and used instead of the flux jacobian if it is greater than the projection of the flux jacobian onto the boundary surface normal. Defaults to "none" 
7. "fixedLowerBound" = lower bound used when no lower bound variable is given. Defaults to 0.

When the row/column variables are defined on the staggered grid, the boundary cell face is shifted to the regular cell boundary (this is done separately in the geometry object). This way the "control volume" of the boundary staggered cells covers the regular grid.

#### 6. Diffusion stencil 

Represents a diffusion term stencil on the regular grid with the option to define the diffusion coefficient using a derivation object. The following "stencilData" options are used by this stencil:

1. "ruleName" = name of the derivation to be used for the diffusion coefficient. Defaults to "none"
2. "requredVarNames" = names of the variables required for the derivation.

### Preparing some common stencils

A number of stencils are used in multiple terms, those are prepared here

In [12]:
# Central differenced divergence stencil
divStencil = sc.centralDiffStencilDiv()


# Upwinded divergence stencil with electron speed as the flux jacobian (constructing the flux as ne * ue)
divStencil_ue = sc.upwindedDiv("ue")

# Similarly for ions 

divStencil_ui = sc.upwindedDiv("ui")

# Left electron flux boundary term with Bohm condition outflow

boundaryStencilLeft_ue = sc.boundaryStencilDiv("ue","cs",isLeft=True) 

# Left ion flux boundary term with Bohm condition outflow

boundaryStencilLeft_ui = sc.boundaryStencilDiv("ui","cs",isLeft=True) 

# Right boundary flux stencils

# Electrons

boundaryStencilRight_ue = sc.boundaryStencilDiv("ue","cs") 

# Ions

boundaryStencilRight_ui = sc.boundaryStencilDiv("ui","cs") 


# Central differenced gradient term

gradStencil = sc.centralDiffStencilGrad()

# Left boundary condition with extrapolation (used for gradient-like operators)

boundaryStencilLeft_grad = sc.boundaryStencilGrad(isLeft=True)

# Right boundary condition with extrapolation (used for gradient-like operators)

boundaryStencilRight_grad = sc.boundaryStencilGrad()

# Simple diagonal stencil

diagonalStencil = sc.diagonalStencil()

### Building electron and ion continuity equation advection models

$\frac{\partial n_e}{\partial t} = - \nabla \cdot (u_e n_e)$ with $|u_e| > c_s$ at boundaries (Bohm boundary condition). Implicit in $n_e$.

In [13]:
#Electron continuity advection

#Adding the model tag to tag list
modelTag = "continuity-ne"

#Initializing model using common models 
electronContModel = cm.collocatedAdvection(modelTag=modelTag
                                           ,advectedVar="ne"
                                           ,advectionSpeed="ue"
                                           ,lowerBoundVar="cs"
                                           ,leftOutflow=True
                                           ,rightOutflow=True)

rk.addModel(electronContModel.dict())

$\frac{\partial n_i}{\partial t} = - \nabla \cdot (u_i n_i)$ with $|u_i| > c_s$ at boundaries (Bohm boundary condition). Implicit in $n_i$.

In [14]:
#Ion continuity advection

#Adding the model tag to tag list
modelTag = "continuity-ni"

#Initializing model using common models
ionContModel = cm.collocatedAdvection(modelTag=modelTag
                                        ,advectedVar="ni"
                                        ,advectionSpeed="ui"
                                        ,lowerBoundVar="cs"
                                        ,leftOutflow=True
                                        ,rightOutflow=True)

rk.addModel(ionContModel.dict())

### Building electron and ion momentum equation advection models

$\frac{\partial \Gamma_e}{\partial t} = - \nabla \cdot (u_e \Gamma_e)$ with $|u_e| > c_s$ at boundaries (Bohm boundary condition). Implicit in $\Gamma_e$.

In [15]:
#Electron momentum advection

#Adding the model tag to tag list
modelTag = "advection-Ge"

#Initializing model
electronMomAdvModel = cm.collocatedAdvection(modelTag=modelTag
                                        ,advectedVar="Ge"
                                        ,advectionSpeed="ue"
                                        ,lowerBoundVar="cs"
                                        ,leftOutflow=True
                                        ,rightOutflow=True)

rk.addModel(electronMomAdvModel.dict())

$\frac{\partial \Gamma_i}{\partial t} = - \nabla \cdot (u_i \Gamma_i)$ with $|u_i| > c_s$ at boundaries (Bohm boundary condition). Implicit in $\Gamma_i$.

In [16]:
#Ion momentum advection

#Adding the model tag to tag list
modelTag = "advection-Gi"

#Initializing model
ionMomAdvModel = cm.collocatedAdvection(modelTag=modelTag
                                        ,advectedVar="Gi"
                                        ,advectionSpeed="ui"
                                        ,lowerBoundVar="cs"
                                        ,leftOutflow=True
                                        ,rightOutflow=True)

rk.addModel(ionMomAdvModel.dict())

### Building electron and ion pressure gradient models in momentum equations

$\frac{\partial \Gamma_e}{\partial t} = - \frac{1}{m_e} \nabla (n_ekT_e) $ with pressure extrapolated at the boundaries. Implicit in $n_e$.

In [17]:
#Electron pressure grad

#Adding the model tag to tag list
modelTag = "pressureGrad-Ge"
#Initializing model
electronPressureGradModel = cm.collocatedPressureGrad(modelTag=modelTag,fluxVar="Ge",densityVar="ne",temperatureVar="Te",speciesMass=elMass)

rk.addModel(electronPressureGradModel.dict())

$\frac{\partial \Gamma_i}{\partial t} = - \frac{1}{m_i} \nabla (n_ikT_i) $ with pressure extrapolated at the boundaries. Implicit in $n_i$.

In [18]:
#Ion pressure grad

#Adding the model tag to tag list
modelTag = "pressureGrad-Gi"

#Initializing model
ionPressureGradModel = cm.collocatedPressureGrad(modelTag=modelTag,fluxVar="Gi",densityVar="ni",temperatureVar="Ti",speciesMass=ionMass)

rk.addModel(ionPressureGradModel.dict())

### Building Ampere-Maxwell equation for E field

In [19]:
#Ampere-Maxwell E field equation 
 
#Adding the model tag to tag list
modelTag = "ampereMaxwell"

#Initializing model
ampereMawellModel = sc.CustomModel(modelTag=modelTag)

# Setting normalization constant calculation 
normConstEl = sc.CustomNormConst(multConst=elCharge/epsilon0,normNames=["density","time","speed","EField"],normPowers=[1.0,1.0,1.0,-1.0])
normConstIon = sc.CustomNormConst(multConst=-elCharge/epsilon0,normNames=["density","time","speed","EField"],normPowers=[1.0,1.0,1.0,-1.0])

evolvedVar="E"

#Electron current term
implicitVar = "Ge"

electronCurrentTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstEl,stencilData=diagonalStencil)

ampereMawellModel.addTerm("electronCurrent",electronCurrentTerm)

#Electron current term
implicitVar = "Gi"

ionCurrentTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstIon,stencilData=diagonalStencil)

ampereMawellModel.addTerm("ionCurrent",ionCurrentTerm)

rk.addModel(ampereMawellModel.dict())

### Building electron and ion momentum equation Lorentz force

$\frac{\partial \Gamma_e}{\partial t} = -en_eE/m_e$ where $E$ is the implicit variable.

Similarly for ions: $\frac{\partial \Gamma_i}{\partial t} = en_iE/m_i$

In [20]:
#Lorentz force terms 
 
#Adding the model tag to tag list
modelTag = "lorentzForce"

#Initializing model
lorentzForceModel = sc.CustomModel(modelTag=modelTag)

# Setting normalization constant calculation 
normConstEl = sc.CustomNormConst(multConst=-elCharge/elMass,normNames=["EField","time","speed"],normPowers=[1.0,1.0,-1.0])
normConstIon = sc.CustomNormConst(multConst=elCharge/ionMass,normNames=["EField","time","speed"],normPowers=[1.0,1.0,-1.0])

vDataEl = sc.VarData(reqRowVars=["ne"])
vDataIon = sc.VarData(reqRowVars=["ni"])
implicitVar="E"

#Electron term
evolvedVar = "Ge"

electronTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstEl,stencilData=diagonalStencil,varData=vDataEl)

lorentzForceModel.addTerm("electronTerm",electronTerm)

#Electron current term
evolvedVar = "Gi"

ionTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstIon,stencilData=diagonalStencil,varData=vDataIon)

lorentzForceModel.addTerm("ionTerm",ionTerm)

rk.addModel(lorentzForceModel.dict())

### Variable-like modelbound data 

The simplest type of modelbound data available for custom fluid models is variable-like modelbound data (selected as "varlikeData"), which has the following properties

- "dataNames" - list of data names in modelbound data object, used as tags to specify the individual data options:
    1. "isDistribution" - set to true if the variable-like data is a distribution. Defaults to false
    2. "isScalar" - set to true if the variable-like data is a scalar. Defaults to false
    3. "isSingleHarmonic" - set to true if variable-like data is a function of x and v, but not a full distribution with multiple harmonics. Defaults to false 
    4. "isDerivedFromOtherData" - set to true if this varlike-data is derived from data in the modelbound data object and not from variables. Defaults to false
    5. "derivationPriority" - sets derivation priority for this data. Defaults to 0
    6. "ruleName" - derivation rule name for the individual varlike-data. Must be supplied
    7. "requiredVarNames" - names of variables required for the derivation

Note that only one of 1-3 above can be true for any individual data entry (all false for a fluid quantity). 

This type of data will be used to construct collision-related moment models below. 

A support class is included in simple_containers.py to streamline the creation of varlike modelbound data.

### Prepare transport coefficient calculations

Based on reduced version of [Makarov et al](https://doi.org/10.1063/5.0047618)

In [21]:
ionZ = 1
sqrt2 = np.sqrt(2)

delta = (1 + 65*sqrt2/32 + 433*sqrt2/288 - 23*sqrt2/16)*ionZ + (5629/1152 - 529/128)*ionZ**2 #A30 in Makarov assuming single ion species and 0 mass ratio

thermFrictionConst = 25*sqrt2*ionZ*(1+11*sqrt2*ionZ/30)/(16*delta) #A50

frictionConst = (1+61*sqrt2*ionZ/72+2*ionZ/9)/delta #A59

elCondConst = 125*(1+433*sqrt2*ionZ/360)/(32*delta)
ionCondConst = 125/32

# Get the e-i Coulomb Log calculated at normalization values 

refZ = rk.normalization["referenceIonZ"]
densNorm = rk.normalization["density"]
tempNorm = rk.normalization["eVTemperature"]


#Normalized e-i coulomb log from NRL formulary
logNorm = 23-np.log(np.sqrt(densNorm*1.0e-6)*refZ*tempNorm**(-1.5)) if tempNorm<10*refZ**2 else 24-np.log(np.sqrt(densNorm*1.0e-6)/tempNorm)

### Building electron-ion friction and Joule heating

Electrons: $m_e\frac{\partial \Gamma_e}{\partial t} = -R_T - R_u = R_{ei}$

Ions: $m_i\frac{\partial \Gamma_i}{\partial t} = R_T + R_u$ 

$R_T = 0.711 n_e \nabla (kT_e)$ where the coefficient is thermFrictionConst above. Implicit in $n_e$

$R_u = \nu_{ei} n_e (u_e - u_i)$ where the collision frequency is related to the default normalization using normalized Coulomb log(see below). Results in two terms for each species.

Joule heating is added to the electron energy equation as

$\frac{\partial W_e}{\partial t} = - R_{ei}(u_e - u_i)$ by evaluating the electron friction terms in the model. Results in two terms.


In [22]:
#Electron-ion friction force terms 
 
#Adding the model tag to tag list
modelTag = "eiFriction"

#Initializing model
eiFrictionModel = sc.CustomModel(modelTag=modelTag)

# Setting normalization constant calculation 
normConstTel = sc.CustomNormConst(multConst=-elCharge*thermFrictionConst/elMass,normNames=["eVTemperature","time","speed","length"],normPowers=[1.0,1.0,-1.0,-1.0])
normConstTion = sc.CustomNormConst(multConst=elCharge*thermFrictionConst/ionMass,normNames=["eVTemperature","time","speed","length"],normPowers=[1.0,1.0,-1.0,-1.0])

#Numerical factors below come from ReMKiT1D to Braginskii time norm conversion (here the fact that time is normalized to the ei collision time is explicitly used)
normConstUelPlus= sc.CustomNormConst(multConst=4/(3*np.sqrt(np.pi)*logNorm))
normConstUelMinus= sc.CustomNormConst(multConst=-4/(3*np.sqrt(np.pi)*logNorm))
normConstUionPlus= sc.CustomNormConst(multConst=4/(3*np.sqrt(np.pi)*logNorm)*elMass/ionMass)
normConstUionMinus= sc.CustomNormConst(multConst=-4/(3*np.sqrt(np.pi)*logNorm)*elMass/ionMass)
normConstJoulePlus=sc.CustomNormConst(multConst=elMass/elCharge,normNames=["speed","eVTemperature"],normPowers=[2.0,-1.0])
normConstJouleMinus=sc.CustomNormConst(multConst=-elMass/elCharge,normNames=["speed","eVTemperature"],normPowers=[2.0,-1.0])

# Creating modelbound data properties for e-i Coulomb log and electron temperature gradient
mbData = sc.VarlikeModelboundData()
mbData.addVariable("logLei",sc.derivationRule("logLeiD+",["Te","ne"]))
mbData.addVariable("gradT",sc.derivationRule("gradDeriv",["Te"]))

eiFrictionModel.setModelboundData(mbData.dict())

vDataGradT = sc.VarData(reqMBRowVars=["gradT"])
#Req vars for the R_u terms include implicit conversion of flux to speed
vDataUEl = sc.VarData(reqRowVars=["ne","Te"],reqRowPowers=[1.0,-1.5],reqMBRowVars=["logLei"])
vDataUIon = sc.VarData(reqRowVars=["ne","Te","ni"],reqRowPowers=[2.0,-1.5,-1.0],reqMBRowVars=["logLei"]) 
vDataJouleEl = sc.VarData(reqRowVars=["ne"],reqRowPowers=[-1.0])
vDataJouleIon = sc.VarData(reqRowVars=["ni"],reqRowPowers=[-1.0])


#Grad T terms 
implicitVar="ne"
evolvedVar = "Ge"

electronGradTFriction = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstTel,varData=vDataGradT,stencilData=diagonalStencil)

eiFrictionModel.addTerm("electronGradTFriction",electronGradTFriction)

evolvedVar = "Gi"

ionGradTFriction = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstTion,varData=vDataGradT,stencilData=diagonalStencil,implicitGroups=[2])

eiFrictionModel.addTerm("ionGradTFriction",ionGradTFriction)

#Electron friction terms
evolvedVar = "Ge"

implicitVar = "Ge"

electronUFrictionA = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstUelMinus,varData=vDataUEl,stencilData=diagonalStencil)

eiFrictionModel.addTerm("eFriction_ue",electronUFrictionA)

implicitVar = "Gi"

electronUFrictionB = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstUelPlus,varData=vDataUIon,stencilData=diagonalStencil)

eiFrictionModel.addTerm("eFriction_ui",electronUFrictionB)

#Ion friction terms 

evolvedVar = "Gi"

implicitVar = "Ge"

ionUFrictionA = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstUionPlus,varData=vDataUEl,stencilData=diagonalStencil,implicitGroups=[2])

eiFrictionModel.addTerm("iFriction_ue",ionUFrictionA)

implicitVar = "Gi"

ionFrictionB = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstUionMinus,varData=vDataUIon,stencilData=diagonalStencil,implicitGroups=[2])

eiFrictionModel.addTerm("iFriction_ui",ionFrictionB)

#Joule heating term

#Other electron friction terms have automatically been put in implicit term group 1, so we need to evaluate that group 

evolvedVar = "We"

implicitVar = "Ge"

jouleHeatingA = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstJouleMinus,evaluatedTermGroup=1,varData=vDataJouleEl,stencilData=diagonalStencil,implicitGroups=[3])

eiFrictionModel.addTerm("jouleHeating_ue",jouleHeatingA)

implicitVar = "Gi"

jouleHeatingB = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstJoulePlus,evaluatedTermGroup=1,varData=vDataJouleIon,stencilData=diagonalStencil,implicitGroups=[3])

eiFrictionModel.addTerm("jouleHeating_ui",jouleHeatingB)

rk.addModel(eiFrictionModel.dict())

### Building implicit temperature derivation

Calculating temperature as an implicit stationary variable allows us to use the natural implicit variable for Braginskii heat flux.

Effectively adds the following equation 

$0 = -kT + \frac{2W}{3n} - \frac{mu^2}{3}$ with $n$,$u$,$T$, and $W$ corresponding to either the electron or ion quantities.

In [23]:
# Implicit temperature equations

#Adding the model tag to tag list
modelTag = "implicitTemp"

#Initializing model
implicitTempModel = sc.CustomModel(modelTag=modelTag)

# Setting normalization constant calculation 
normConstI = sc.CustomNormConst(multConst=-1.0)
normConstW = sc.CustomNormConst(multConst=2/3)
normConstU2El = sc.CustomNormConst(multConst=-elMass/(3*elCharge),normNames=["speed","eVTemperature"],normPowers=[2.0,-1.0])
normConstU2Ion = sc.CustomNormConst(multConst=-ionMass/(3*elCharge),normNames=["speed","eVTemperature"],normPowers=[2.0,-1.0])

vDataWEl = sc.VarData(reqRowVars=["ne"],reqRowPowers=[-1.0])
vDataWIon = sc.VarData(reqRowVars=["ni"],reqRowPowers=[-1.0])

#Kinetic energy term will be implicit in fluxes so need to be converted to speeds
vDataU2El = sc.VarData(reqRowVars=["ne","Ge"],reqRowPowers=[-2.0,1.0])
vDataU2Ion = sc.VarData(reqRowVars=["ni","Gi"],reqRowPowers=[-2.0,1.0])

# Electrons 

evolvedVar = "Te"

# Identity term

identityTermEl = sc.GeneralMatrixTerm(evolvedVar,customNormConst=normConstI,stencilData=diagonalStencil)

implicitTempModel.addTerm("identityTerm_e",identityTermEl)

# 2/3 W/n term 

termWEl = sc.GeneralMatrixTerm(evolvedVar,implicitVar="We",customNormConst=normConstW,varData=vDataWEl,stencilData=diagonalStencil)

implicitTempModel.addTerm("wTerm_e",termWEl)

# kinetic energy term 

termU2El = sc.GeneralMatrixTerm(evolvedVar,implicitVar="Ge",customNormConst=normConstU2El,varData=vDataU2El,stencilData=diagonalStencil)

implicitTempModel.addTerm("u2Term_e",termU2El)

#Ions

evolvedVar = "Ti"

# Identity term

identityTermIons = sc.GeneralMatrixTerm(evolvedVar,customNormConst=normConstI,stencilData=diagonalStencil)

implicitTempModel.addTerm("identityTerm_i",identityTermIons)

# 2/3 W/n term 

termWIon = sc.GeneralMatrixTerm(evolvedVar,implicitVar="Wi",customNormConst=normConstW,varData=vDataWIon,stencilData=diagonalStencil)

implicitTempModel.addTerm("wTerm_i",termWIon)

# kinetic energy term 

termU2Ion = sc.GeneralMatrixTerm(evolvedVar,implicitVar="Gi",customNormConst=normConstU2Ion,varData=vDataU2Ion,stencilData=diagonalStencil)

implicitTempModel.addTerm("u2Term_i",termU2Ion)

#Updating model properties based on constructed model
rk.addModel(implicitTempModel.dict())

### Building electron and ion energy advection 

$\frac{\partial W_e}{\partial t} = - \nabla \cdot (u_e W_e)$ with reflective boundaries so all heatflux BCs can go into sheath heat transmission coeff. Implicit in $W_e$.

In [24]:
#Electron energy advection

#Adding the model tag to tag list
modelTag = "advection-We"

#Initializing model
electronWAdvection = cm.collocatedAdvection(modelTag=modelTag
                                           ,advectedVar="We"
                                           ,advectionSpeed="ue")

# No boundary terms means reflective boundaries => allows all outflow to be governed by sheath heat transmission coefficients 

rk.addModel(electronWAdvection.dict())

$\frac{\partial W_i}{\partial t} = - \nabla \cdot (u_i W_i)$ with reflective boundaries so all heatflux BCs can go into sheath heat transmission coeff. Implicit in $W_i$.

In [25]:
#Ion energy advection

#Adding the model tag to tag list
modelTag = "advection-Wi"

#Initializing model
ionWAdvection = cm.collocatedAdvection(modelTag=modelTag
                                           ,advectedVar="Wi"
                                           ,advectionSpeed="ui")

rk.addModel(ionWAdvection.dict())

### Building electron and ion pressure advection terms in energy equations

$\frac{\partial W_e}{\partial t} = - \nabla \cdot (n_ekT_eu_e)$ with reflective boundary conditions for same reason as above.

In [26]:
#Electron pressure advection

#Adding the model tag to tag list
modelTag = "advection-pe"

#Initializing model
electronPAdvection = sc.CustomModel(modelTag=modelTag)

# Setting normalization constant calculation (here -u_0*t_0/x_0 - should be -1, names and powers included for demonstration purposes)
normConst = sc.CustomNormConst(multConst=-1.0,normNames=["speed","time","length"],normPowers=[1.0,1.0,-1.0])

vData = sc.VarData(reqColVars=["Te"])

evolvedVar="We"
implicitVar="ne"
divFluxTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConst,stencilData=divStencil_ue,varData=vData)

electronPAdvection.addTerm("divFlux",divFluxTerm)

# No boundary terms means reflective boundaries => allows all outflow to be governed by sheath heat transmission coefficients 

rk.addModel(electronPAdvection.dict())

$\frac{\partial W_i}{\partial t} = - \nabla \cdot (n_ikT_iu_i)$ with reflective boundary conditions due to for same reason as above.

In [27]:
#Ion pressure advection

#Adding the model tag to tag list
modelTag = "advection-pi"

#Initializing model
ionPAdvection = sc.CustomModel(modelTag=modelTag)

# Setting normalization constant calculation (here -u_0*t_0/x_0 - should be -1, names and powers included for demonstration purposes)
normConst = sc.CustomNormConst(multConst=-1.0,normNames=["speed","time","length"],normPowers=[1.0,1.0,-1.0])

vData = sc.VarData(reqColVars=["Ti"])

evolvedVar="Wi"
implicitVar="ni"
divFluxTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConst,stencilData=divStencil_ui,varData=vData)

ionPAdvection.addTerm("divFlux",divFluxTerm)

# No boundary terms means reflective boundaries => allows all outflow to be governed by sheath heat transmission coefficients 

rk.addModel(ionPAdvection.dict())

### Building Lorentz force work terms

$\frac{\partial W_e}{\partial t} = -e\Gamma_e E$ for electrons. Implicit in $E$.

Similarly, for ions: $\frac{\partial W_i}{\partial t} = e\Gamma_i E$

In [28]:
# Lorentz force work terms

#Adding the model tag to tag list
modelTag = "lorentzForceWork"

#Initializing model
lorentzForceWorkModel = sc.CustomModel(modelTag=modelTag)

# Setting normalization constant calculation 
normConstEl = sc.CustomNormConst(multConst=-1.0,normNames=["EField","time","speed","eVTemperature"],normPowers=[1.0,1.0,1.0,-1.0])
normConstIon = sc.CustomNormConst(normNames=["EField","time","speed","eVTemperature"],normPowers=[1.0,1.0,1.0,-1.0])

vDataEl = sc.VarData(reqRowVars=["Ge"])
vDataIon = sc.VarData(reqRowVars=["Gi"])
implicitVar="E"

#Electron term
evolvedVar = "We"

electronTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstEl,stencilData=diagonalStencil,varData=vDataEl)

lorentzForceWorkModel.addTerm("electronTerm",electronTerm)

#Electron current term
evolvedVar = "Wi"

ionTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstIon,stencilData=diagonalStencil,varData=vDataIon)

lorentzForceWorkModel.addTerm("ionTerm",ionTerm)

rk.addModel(lorentzForceWorkModel.dict())

### Building stationary Braginskii heat fluxes

The Braginskii heat fluxes are constructed as stationary equations of the form:

Electrons: $0 = -q_e + q_{T_e} + q_u$

Ions: $0 = - q_i + q_{T_i}$

Where $q_{T_s} = \kappa_s \nabla (kT_s)$ and $q_u = -0.711kT_e(u_e - u_i)$. The gradient is calculated implicitly and extrapolates the temperature to the boundary.

$\kappa_s$ is proportional to $T_s^{5/2}$ and is calculated using the elCondConst and ionCondConst defined above.

In [29]:
# Braginskii heat fluxes

#Adding the model tag to tag list
modelTag = "braginskiiq"

#Initializing model
braginskiiHFModel = sc.CustomModel(modelTag=modelTag)

# Creating modelbound data properties for e-e and i-i Coulomb logs

mbData = sc.VarlikeModelboundData()

mbData.addVariable("logLee",sc.derivationRule("logLee",["Te","ne"]))
mbData.addVariable("logLii",sc.derivationRule("logLiiD+_D+",["ni","ni","Ti","Ti"]))

braginskiiHFModel.setModelboundData(mbData.dict())

# Setting normalization constant calculation 
normConstI = sc.CustomNormConst(multConst=-1.0)

nConstGradT = 12*np.pi**1.5*epsilon0**2/np.sqrt(elMass*elCharge) # Comes from e-i collision time

normConstGradTEl = sc.CustomNormConst(multConst=-nConstGradT*elCondConst,normNames=["eVTemperature","length","heatFlux"],normPowers=[3.5,-1.0,-1.0])
normConstGradTIon = sc.CustomNormConst(multConst=-nConstGradT*ionCondConst*np.sqrt(elMass/ionMass),normNames=["eVTemperature","length","heatFlux"],normPowers=[3.5,-1.0,-1.0])

normConstUPlus = sc.CustomNormConst(multConst=elCharge*thermFrictionConst,normNames=["density","eVTemperature","speed","heatFlux"],normPowers=[1.0,1.0,1.0,-1.0])
normConstUMinus = sc.CustomNormConst(multConst=-elCharge*thermFrictionConst,normNames=["density","eVTemperature","speed","heatFlux"],normPowers=[1.0,1.0,1.0,-1.0])

#Variable data 

gradDataEl = sc.VarData(reqRowVars=["Te"],reqRowPowers=[2.5],reqMBRowVars=["logLee"],reqMBRowPowers=[-1.0])
gradDataIon = sc.VarData(reqRowVars=["Ti"],reqRowPowers=[2.5],reqMBRowVars=["logLii"],reqMBRowPowers=[-1.0])

uDataA = sc.VarData(reqRowVars=["Te"])
uDataB = sc.VarData(reqRowVars=["ne","Te","ni"],reqRowPowers=[1.0,1.0,-1.0])

# Electrons 

evolvedVar = "qe"

#Identity term

identityTermEl = sc.GeneralMatrixTerm(evolvedVar,customNormConst=normConstI,stencilData=diagonalStencil)

braginskiiHFModel.addTerm("identityTerm_e",identityTermEl)

#Gradient terms 

implicitVar = "Te"

gradTermEl = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstGradTEl,stencilData=gradStencil,varData=gradDataEl)

braginskiiHFModel.addTerm("bulkGrad_e",gradTermEl)

# Add left boundary term 

leftBCTermEl = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstGradTEl,stencilData=boundaryStencilLeft_grad,varData=gradDataEl)

braginskiiHFModel.addTerm("leftBC_e",leftBCTermEl)

# Add Right boundary term 

rightBCTermEl = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstGradTEl,stencilData=boundaryStencilRight_grad,varData=gradDataEl)

braginskiiHFModel.addTerm("rightBC_e",rightBCTermEl)

# qu terms for electrons

implicitVar = "Ge"

electronUHFA = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstUMinus,varData=uDataA,stencilData=diagonalStencil)

braginskiiHFModel.addTerm("qu_ue",electronUHFA)

implicitVar = "Gi"

electronUHFB = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstUPlus,varData=uDataB,stencilData=diagonalStencil)

braginskiiHFModel.addTerm("qu_ui",electronUHFB)

# Ions

evolvedVar = "qi"

#Identity term

identityTermIon = sc.GeneralMatrixTerm(evolvedVar,customNormConst=normConstI,stencilData=diagonalStencil)

braginskiiHFModel.addTerm("identityTerm_i",identityTermIon)

#Gradient terms 

implicitVar = "Ti"

gradTermIon = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstGradTIon,stencilData=gradStencil,varData=gradDataIon)

braginskiiHFModel.addTerm("bulkGrad_i",gradTermIon)

# Add left boundary term 

leftBCTermIon = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstGradTIon,stencilData=boundaryStencilLeft_grad,varData=gradDataIon)

braginskiiHFModel.addTerm("leftBC_i",leftBCTermIon)

# Add Right boundary term 

rightBCTermIon = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstGradTIon,stencilData=boundaryStencilRight_grad,varData=gradDataIon)

braginskiiHFModel.addTerm("rightBC_i",rightBCTermIon)

rk.addModel(braginskiiHFModel.dict())


### Building heat flux divergence terms with sheath boundary conditions 

$\frac{\partial W_e}{\partial t} = - \nabla \cdot q_e$ with the boundary condition given by $q_{sh} = \gamma_e kT_en_e u_{e,sh}$ where $|u_{e,sh}| > c_s$ and $\gamma_e$ is a derived scalar quantity define above.

In [30]:
# Electron heat flux divergence 

#Adding the model tag to tag list
modelTag = "divq_e"

#Initializing model
electronDivQModel = sc.CustomModel(modelTag=modelTag)

#Setting normalization constants

normFlux = sc.CustomNormConst(multConst=-1/elCharge,normNames=["heatFlux","time","length","density","eVTemperature"],normPowers=[1.0,1.0,-1.0,-1.0,-1.0])
normBC = sc.CustomNormConst(multConst=-1.0,normNames=["speed","time","length"],normPowers=[1.0,1.0,-1.0])

vDataBCLeft = sc.VarData(reqRowVars=["gammaLeft"],reqColVars=["Te"])
vDataBCRight = sc.VarData(reqRowVars=["gammaRight"],reqColVars=["Te"])

#Bulk flux divergence 

evolvedVar = "We"
implicitVar = "qe"

divFluxTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normFlux,stencilData=divStencil)

electronDivQModel.addTerm("divFlux",divFluxTerm)

# Add left boundary term with Bohm condition to outflow

implicitVar = "ne"

leftBCTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normBC,varData=vDataBCLeft,stencilData=boundaryStencilLeft_ue)

electronDivQModel.addTerm("leftBC",leftBCTerm)

# Add Right boundary term with Bohm condition to outflow

rightBCTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normBC,varData=vDataBCRight,stencilData=boundaryStencilRight_ue)

electronDivQModel.addTerm("rightBC",rightBCTerm)

rk.addModel(electronDivQModel.dict())

$\frac{\partial W_i}{\partial t} = - \nabla \cdot q_i$ with the boundary condition given by $q_{sh} = \gamma_i kT_in_i u_{i,sh}$ where $|u_{i,sh}| > c_s$ and $\gamma_i$ is a derived scalar quantity define above (constant 2.5 here).

In [31]:
# Ion heat flux divergence 

#Adding the model tag to tag list
modelTag = "divq_i"

#Initializing model
ionDivQModel = sc.CustomModel(modelTag=modelTag)

#Setting normalization constants

normFlux = sc.CustomNormConst(multConst=-1/elCharge,normNames=["heatFlux","time","length","density","eVTemperature"],normPowers=[1.0,1.0,-1.0,-1.0,-1.0])
normBC = sc.CustomNormConst(multConst=-1.0,normNames=["speed","time","length"],normPowers=[1.0,1.0,-1.0])

vDataBC = sc.VarData(reqRowVars=["ionGamma"],reqColVars=["Ti"])

#Bulk flux divergence 

evolvedVar = "Wi"
implicitVar = "qi"

divFluxTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normFlux,stencilData=divStencil)

ionDivQModel.addTerm("divFlux",divFluxTerm)

# Add left boundary term with Bohm condition to outflow

implicitVar = "ni"

leftBCTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normBC,varData=vDataBC,stencilData=boundaryStencilLeft_ui)

ionDivQModel.addTerm("leftBC",leftBCTerm)

# Add Right boundary term with Bohm condition to outflow

rightBCTerm = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normBC,varData=vDataBC,stencilData=boundaryStencilRight_ui)

ionDivQModel.addTerm("rightBC",rightBCTerm)

rk.addModel(ionDivQModel.dict())

### Building electron-ion heat exchange 

$\frac{\partial W_e}{\partial t} = - 3 \frac{m_e}{m_i} \frac{n_e}{\tau_{ei}}(T_e-T_i)$ where $\tau_{ei}$ is the electron-ion Coulomb collision time. 

Similarly for ions, with opposite sign.

In [32]:
# Electron-ion heat exchange terms

#Adding the model tag to tag list
modelTag = "eiHeatEx"

#Initializing model
eiHeatExModel = sc.CustomModel(modelTag=modelTag)

normConstPlus = sc.CustomNormConst(multConst=4/np.sqrt(np.pi)*elMass/ionMass) # Numerical factor from conversion of ReMKiT1D to Braginskii collision time
normConstMinus = sc.CustomNormConst(multConst=-4/np.sqrt(np.pi)*elMass/ionMass)

# Creating modelbound data properties for e-i Coulomb log 

mbData = sc.VarlikeModelboundData()
mbData.addVariable("logLei",sc.derivationRule("logLeiD+",["Te","ne"]))

eiHeatExModel.setModelboundData(mbData.dict())

vData = sc.VarData(reqRowVars=["ne","Te"],reqRowPowers=[2.0,-1.5],reqMBRowVars=["logLei"])

#Electron terms 

evolvedVar = "We"

implicitVar = "Te"

heatExTermAEl = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstMinus,varData=vData,stencilData=diagonalStencil)

eiHeatExModel.addTerm("heatExTermA_e",heatExTermAEl)

implicitVar = "Ti"

heatExTermBEl = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstPlus,varData=vData,stencilData=diagonalStencil)

eiHeatExModel.addTerm("heatExTermB_e",heatExTermBEl)

#Ion terms

evolvedVar = "Wi"

implicitVar = "Ti"

heatExTermAIon = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstMinus,varData=vData,stencilData=diagonalStencil,implicitGroups=[2])

eiHeatExModel.addTerm("heatExTermA_i",heatExTermAIon)

implicitVar = "Te"

heatExTermBIon = sc.GeneralMatrixTerm(evolvedVar,implicitVar=implicitVar,customNormConst=normConstPlus,varData=vData,stencilData=diagonalStencil,implicitGroups=[2])

eiHeatExModel.addTerm("heatExTermB_i",heatExTermBIon)

rk.addModel(eiHeatExModel.dict())

### Building particle sources

Simple box particle source in centre of domain

$\frac{\partial n_e}{\partial t} = S_n$ Implicit in density, but density dependence removed by additional row variable (this allows for easy "implicit" implementation). 

Same for ions.

In [33]:
particleInjectionRate = 0.02
xProfile = np.zeros(128)
xProfile[54:74] = particleInjectionRate/(20*1.28)

In [34]:
# Particle source model

#Adding the model tag to tag list
modelTag = "particleSource"

#Initializing model
particleSourceModel = sc.CustomModel(modelTag=modelTag)

#Electrons

evolvedVar = "ne"
particleSourceTermEl = cm.simpleSourceTerm(evolvedVar=evolvedVar,sourceProfile=xProfile)

particleSourceModel.addTerm("electronSource",particleSourceTermEl)

#Ions 
evolvedVar = "ni"
particleSourceTermIon = cm.simpleSourceTerm(evolvedVar=evolvedVar,sourceProfile=xProfile)

particleSourceModel.addTerm("ionSource",particleSourceTermIon)

rk.addModel(particleSourceModel.dict())

### Building energy sources

Energy source at centre of domain with Gaussian shape.

$\frac{\partial W_e}{\partial t} = S_W$ and similarly for ions.

In [35]:
energyInjectionRate = 0.04
xProfileEnergy = energyInjectionRate*np.exp((-np.power(gridObj.xGrid - np.mean(gridObj.xGrid), 2.) / (2 * np.power(1.5, 2.))))


In [36]:
# Energy source model

#Adding the model tag to tag list
modelTag = "energySource"

#Initializing model
energySourceModel = sc.CustomModel(modelTag=modelTag)

#Electrons

evolvedVar = "We"
energySourceTermEl = cm.simpleSourceTerm(evolvedVar=evolvedVar,sourceProfile=xProfileEnergy)

energySourceModel.addTerm("electronSource",energySourceTermEl)

#Ions 
evolvedVar = "Wi"
energySourceTermIon = cm.simpleSourceTerm(evolvedVar=evolvedVar,sourceProfile=xProfileEnergy)

energySourceModel.addTerm("ionSource",energySourceTermIon)

rk.addModel(energySourceModel.dict())

Particle source at local temperature

In [37]:
xProfileEnergyS = 3*xProfile/2

In [38]:
# Energy source model making sure particles are injected with the local temperature

#Adding the model tag to tag list
modelTag = "energySourceParts"

#Initializing model
energySourceModel = sc.CustomModel(modelTag=modelTag)

#Electrons

evolvedVar = "We"
energySourceTermEl = sc.GeneralMatrixTerm(evolvedVar,implicitVar="Te",spatialProfile=xProfileEnergyS.tolist(),stencilData=diagonalStencil)

energySourceModel.addTerm("electronSource",energySourceTermEl)

#Ions 
evolvedVar = "Wi"
energySourceTermIon = sc.GeneralMatrixTerm(evolvedVar,implicitVar="Ti",spatialProfile=xProfileEnergyS.tolist(),stencilData=diagonalStencil)

energySourceModel.addTerm("ionSource",energySourceTermIon)

rk.addModel(energySourceModel.dict())

### Manipulator options 

Manipulators are flexible objects used to manipulate data outside of standard integration and derivation options. Below is an example of how to create an evaluator type manipulator and use it to access the evaluated electron-ion energy exchange term by evaluation the corresponding model and term group, as well as a modelbound data extractor type to get the  Coulomb log from the same model. 

An important aspect of manipulators is their priority, determining when they will be called: 

0. Manipulator is called on all internal integrator iterations (nonlinear BDE iterations in this case)
1. Manipulator is called at the end of each (internal) timestep (in any one integration step)
2. Manipulator is called at the end of each integration step (here there is only one integration step and no internal step timestep control so 1 and 2 behave the same)
3. Manipulator is called at the end of each integration call (global timestep)
4. Manipulator is called before data writing to HDF5 file

Importantly, whenever a manipulator call is made with a given priority all manipulators with priority <= the call priority will be invoked.

Routines for manipulator dictionary creation can be found in simple_containers.py

In [39]:

evaluator = sc.groupEvaluatorManipulator("eiHeatEx",1,"Qei")

rk.addManipulator("QeiEvaluator",evaluator)

extractor = sc.extractorManipulator("eiHeatEx","logLei","logL")

rk.addManipulator("logLExtractor",extractor)

### Variable diagnosis

In [45]:
rk.getTermsThatEvolveVar("ne")
rk.addTermDiagnosisForVars(["ne","Te"])

In [43]:
rk.manipulatorData

{'tags': ['QeiEvaluator',
  'logLExtractor',
  'continuity-nedivFlux',
  'continuity-neleftBC',
  'continuity-nerightBC',
  'particleSourceelectronSource',
  'implicitTempidentityTerm_e',
  'implicitTempwTerm_e',
  'implicitTempu2Term_e'],
 'QeiEvaluator': {'type': 'groupEvaluator',
  'modelTag': 'eiHeatEx',
  'evaluatedTermGroup': 1,
  'resultVarName': 'Qei',
  'priority': 4},
 'logLExtractor': {'type': 'modelboundDataExtractor',
  'modelTag': 'eiHeatEx',
  'modelboundDataName': 'logLei',
  'resultVarName': 'logL',
  'priority': 4},
 'continuity-nedivFlux': {'type': 'termEvaluator',
  'evaluatedModelNames': ['continuity-ne'],
  'evaluatedTermNames': ['divFlux'],
  'resultVarName': 'continuity-nedivFlux',
  'priority': 4},
 'continuity-neleftBC': {'type': 'termEvaluator',
  'evaluatedModelNames': ['continuity-ne'],
  'evaluatedTermNames': ['leftBC'],
  'resultVarName': 'continuity-neleftBC',
  'priority': 4},
 'continuity-nerightBC': {'type': 'termEvaluator',
  'evaluatedModelNames': [

In [44]:
rk.varList()

['ne',
 'ni',
 'Ge',
 'Gi',
 'We',
 'Wi',
 'qe',
 'qi',
 'E',
 'Te',
 'Ti',
 'ue',
 'ui',
 'cs',
 'Qei',
 'logL',
 'time',
 'gammaLeft',
 'gammaRight',
 'ionGamma',
 'continuity-nedivFlux',
 'continuity-neleftBC',
 'continuity-nerightBC',
 'particleSourceelectronSource',
 'implicitTempidentityTerm_e',
 'implicitTempwTerm_e',
 'implicitTempu2Term_e']

### Integrator options

ReMKiT1D allows for highly customizable integrator options, with the default integrator being a composite integrator object containing Runge-Kutta explicit and Backwards Euler (with Picard iterations) implicit integrators. The properties of individual integrators can be modified and the integrators arranged in integration steps to produce more complicated integration schemes, such as Strang splitting. 

In this example only the Backwards Euler integrator is used, as set up below.

In [46]:
integrator = sc.picardBDEIntegrator(absTol=10.0,convergenceVars=["ne","ni","Ge","Gi","We","Wi"]) 

rk.addIntegrator("BE",integrator)

### Timestep control

There are two default options for step control at the composite integrator level. The standard behaviour is keeping a constant initial timestep, while a timestep controller object can be assigned to the integrator to control the timestep based on some criterion.

The standard behaviour is retained in this example.

The number of allowed implicit and general groups is also set here.

In [47]:
initialTimestep=0.1
rk.setIntegratorGlobalData(3,2,initialTimestep) 

### Controlling integration steps

As mentioned above, ReMKiT1D alows for composing integrators in a sequence using integration steps. 

This example uses the simplest behaviour - a single step integration

In [48]:

bdeStep = sc.IntegrationStep("BE",defaultEvaluateGroups=[1,2,3],defaultUpdateModelData=True,defaultUpdateGroups=[1,2,3])

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

### Time loop options

The main part of ReMKiT1D is the time loop, where the variables are advanced through time by repeatedly calling the integrators defined above. The following shows a way to set timeloop options:

In [49]:
rk.setFixedNumTimesteps(8000)
rk.setFixedStepOutput(500)

### Create config file

In [50]:
rk.writeConfigFile()

### Data analysis

In [51]:
numFiles = 16

In [52]:
loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [53]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames,varsToIgnore=["ionGamma"])
loadedData

<xarray.Dataset>
Dimensions:                       (x: 128, h: 1, v: 80, time: 17, dim_0: 1)
Coordinates:
  * x                             (x) float64 0.05 0.15 0.25 ... 12.65 12.75
  * h                             (h) int64 0
  * v                             (v) float64 0.005 0.01546 ... 104.7 114.3
  * time                          (time) float64 0.0 50.0 100.0 ... 750.0 800.0
Dimensions without coordinates: dim_0
Data variables: (12/26)
    ne                            (time, x) float64 1.0 1.0 ... 0.5225 0.4951
    ni                            (time, x) float64 1.0 1.0 ... 0.5225 0.4951
    Ge                            (time, x) float64 0.0 0.0 ... 0.0191 0.01874
    Gi                            (time, x) float64 0.0 0.0 ... 0.0191 0.01874
    We                            (time, x) float64 1.5 1.5 1.5 ... 4.259 4.034
    Wi                            (time, x) float64 1.5 1.5 1.5 ... 7.765 7.542
    ...                            ...
    continuity-neleftBC           (time, x) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    continuity-nerightBC          (time, x) float64 0.0 0.0 0.0 ... 0.0 -0.02645
    particleSourceelectronSource  (time, x) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    implicitTempidentityTerm_e    (time, x) float64 0.0 0.0 ... -5.433 -5.431
    implicitTempwTerm_e           (time, x) float64 0.0 0.0 0.0 ... 5.434 5.432
    implicitTempu2Term_e          (time, x) float64 0.0 0.0 ... -0.0009552

In [57]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

#### Explore data using basic dashboard

In [58]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,gridObj)

dashboard.fluid2Comparison().show()


Launching server at http://localhost:35625
